<a href="https://colab.research.google.com/github/chandan241/st_tr/blob/main/manual_multiple_select.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install kiteconnect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 771.5/771.5 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.9/264.9 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 6.5 MB/s eta 0:00:00


In [ ]:
import logging
import time
from kiteconnect import KiteConnect
from collections import deque
from IPython.display import clear_output
# keep last 10 messages
log_buffer = deque(maxlen=10)

logging.basicConfig(level=logging.DEBUG)
api_key=""
api_secret = ""
kite = KiteConnect(api_key=api_key)
kite.login_url()

'https://kite.zerodha.com/connect/login?api_key=l9k2jvh835bb37nw&v=3'

In [ ]:
request_token = "LghLSRxGq6MHomRkm0T05nFKIqpfegW5"

In [ ]:
data = kite.generate_session(request_token, api_secret=api_secret)
kite.set_access_token(data["access_token"])

In [ ]:
# @title Cell 1 (Display the options prices) - Multi-select + Top 3 CE/PE buttons + Refresh
import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output

print('**********************CONTROL LOSSES THAN MAKING PROFIT. MARKET WILL GIVE 2ND CHANCE****************************')

INDEX_CONFIG = {
    "NIFTY":  {"segment": "NFO", "name": "NIFTY",  "spot": "NSE:NIFTY 50"},
    "SENSEX": {"segment": "BFO", "name": "SENSEX", "spot": "BSE:SENSEX"},
}

# Global state
checkboxes = {}
ce_list = []            # Ordered list of CE symbols for fast "top 3 CE"
pe_list = []            # Ordered list of PE symbols for fast "top 3 PE"
symbol_meta = {}
current_prefix = None
atm_info = {}
out = widgets.Output()

# --- Widgets ---
index_dropdown = widgets.Dropdown(
    options=[("NIFTY 50", "NIFTY"), ("SENSEX", "SENSEX")],
    value="NIFTY",
    description="Index:",
    style={'description_width': 'initial'}
)

expiry_dropdown = widgets.Dropdown(
    options=[],
    description="Expiry:",
    style={'description_width': 'initial'}
)

btn_top3_ce = widgets.Button(description="Top 3 CE", button_style="info")
btn_top3_pe = widgets.Button(description="Top 3 PE", button_style="info")
btn_clear   = widgets.Button(description="Clear All", button_style="warning")

# NEW: Refresh button
btn_refresh = widgets.Button(description="Refresh", button_style="primary", tooltip="Refresh quotes & ATM")

controls_box = widgets.HBox([index_dropdown, expiry_dropdown, btn_top3_ce, btn_top3_pe, btn_clear, btn_refresh])

display(controls_box, out)

# =========================
# Helper to build checkbox
# =========================
def make_checkbox(sym, strike, label, meta):
    cb = widgets.Checkbox(value=False, description="", layout=widgets.Layout(width="auto"))
    cb.style = {'description_width': '0px'}
    html_label = widgets.HTML(value=label, layout=widgets.Layout(margin="0 0 0 5px"))
    hbox = widgets.HBox([cb, html_label], layout=widgets.Layout(align_items="center"))
    checkboxes[sym] = cb
    symbol_meta[sym] = meta
    return hbox

# =========================
# Load expiries
# =========================
def refresh_expiries(index_key):
    cfg = INDEX_CONFIG[index_key]
    segment = cfg["segment"]
    name = cfg["name"]

    instruments = kite.instruments(segment)

    options = [i for i in instruments if i['name'] == name and i['instrument_type'] in ["CE", "PE"]]
    today = datetime.date.today()
    exps = sorted({i['expiry'] for i in options if i['expiry'] >= today})

    if not exps:
        expiry_dropdown.options = [("No expiries found", None)]
        expiry_dropdown.value = None
        return options, None

    if len(exps) == 1:
        expiry_dropdown.options = [(exps[0].strftime("%d-%b-%Y") + " (Only)", exps[0])]
        expiry_dropdown.value = exps[0]
    else:
        expiry_dropdown.options = [
            ("This Week - " + exps[0].strftime("%d-%b-%Y"), exps[0]),
            ("Next Week - " + exps[1].strftime("%d-%b-%Y"), exps[1]),
        ]
        # preserve existing selection if possible; else default to first
        if expiry_dropdown.value not in exps:
            expiry_dropdown.value = exps[0]

    return options, exps

# =========================
# Render chain
# =========================
def render_chain(index_key, expiry_date, preserve_selection=True):
    global checkboxes, symbol_meta, ce_list, pe_list, current_prefix, atm_info

    # Capture currently selected symbols to preserve (optional)
    previously_selected = set([sym for sym, cb in checkboxes.items() if cb.value]) if preserve_selection else set()

    checkboxes = {}
    symbol_meta = {}
    ce_list = []
    pe_list = []

    cfg = INDEX_CONFIG[index_key]
    segment = cfg["segment"]
    name = cfg["name"]
    spot_symbol = cfg["spot"]
    current_prefix = segment

    instruments = kite.instruments(segment)
    expiry_opts = [
        i for i in instruments
        if i['name'] == name and i['instrument_type'] in ["CE", "PE"] and i['expiry'] == expiry_date
    ]

    if not expiry_opts:
        with out:
            out.clear_output()
            print(f"No options found for {name} on {expiry_date}.")
        return

    # --- Spot & ATM ---
    spot = kite.ltp(spot_symbol)[spot_symbol]["last_price"]
    strikes = sorted({i['strike'] for i in expiry_opts})
    atm_strike = min(strikes, key=lambda s: abs(s - spot))
    atm_info[name] = {"atm_strike": atm_strike, "spot": spot}

    # --- CE & PE OTM selection ---
    ce_otm = sorted([i for i in expiry_opts if i['instrument_type']=="CE" and i['strike'] > atm_strike],
                    key=lambda x: x['strike'])[:10]
    pe_otm = sorted([i for i in expiry_opts if i['instrument_type']=="PE" and i['strike'] < atm_strike],
                    key=lambda x: -x['strike'])[:10]

    ce_list = [opt["tradingsymbol"] for opt in ce_otm]
    pe_list = [opt["tradingsymbol"] for opt in pe_otm]

    symbols = ce_list + pe_list
    quote_data = kite.quote([f"{current_prefix}:{sym}" for sym in symbols]) if symbols else {}

    def get_change(sym):
        key = f"{current_prefix}:{sym}"
        data = quote_data.get(key, {})
        ltp = data.get("last_price")
        prev_close = data.get("ohlc", {}).get("close")

        if ltp is None: return "NA"
        if prev_close:
            pct = ((ltp - prev_close) / prev_close) * 100
            color = "green" if pct >= 0 else "red"
            arrow = "🔺" if pct >= 0 else "🔻"
            return f"{ltp:.2f} (<span style='color:{color};'>{arrow}{pct:.2f}%</span>)"

        return f"{ltp:.2f} (NA)"

    ce_items, pe_items = [], []

    # --- Build CE column ---
    for opt in ce_otm:
        sym = opt["tradingsymbol"]
        label = f"<b>{opt['strike']}</b> | {sym} | {get_change(sym)}"
        meta = {"strike": opt["strike"], "instrument_type": "CE", "expiry": opt["expiry"], "index_name": name}
        cb_row = make_checkbox(sym, opt["strike"], label, meta)
        ce_items.append(cb_row)

    # --- Build PE column ---
    for opt in pe_otm:
        sym = opt["tradingsymbol"]
        label = f"<b>{opt['strike']}</b> | {sym} | {get_change(sym)}"
        meta = {"strike": opt["strike"], "instrument_type": "PE", "expiry": opt["expiry"], "index_name": name}
        cb_row = make_checkbox(sym, opt["strike"], label, meta)
        pe_items.append(cb_row)

    # Restore selections if symbol still present
    if preserve_selection:
        for sym in previously_selected:
            if sym in checkboxes:
                checkboxes[sym].value = True

    atm_label = widgets.HTML(f"<b>{name} ATM Strike: {atm_strike} (Spot: {round(spot,2)})</b>")

    cols = widgets.HBox([
        widgets.VBox([widgets.Label("📈 OTM Calls (CE > ATM)")] + ce_items,
                     layout=widgets.Layout(border="1px solid gray", padding="5px", width="420px")),
        widgets.VBox([widgets.Label("📉 OTM Puts (PE < ATM)")] + pe_items,
                     layout=widgets.Layout(border="1px solid gray", padding="5px", width="420px")),
    ])

    with out:
        out.clear_output()
        display(widgets.VBox([atm_label, cols]))

# =========================
# Button handlers
# =========================
def top3_ce_clicked(b):
    for cb in checkboxes.values():
        cb.value = False  # clear first
    for sym in ce_list[:3]:
        if sym in checkboxes:
            checkboxes[sym].value = True

def top3_pe_clicked(b):
    for cb in checkboxes.values():
        cb.value = False
    for sym in pe_list[:3]:
        if sym in checkboxes:
            checkboxes[sym].value = True

def clear_all_clicked(b):
    for cb in checkboxes.values():
        cb.value = False

# NEW: Refresh handler
def refresh_clicked(b):
    # Option A: fast refresh without touching expiry list
    # render_chain(index_dropdown.value, expiry_dropdown.value, preserve_selection=True)

    # Option B: refresh expiries too (useful around weekly rollovers)
    _, _ = refresh_expiries(index_dropdown.value)
    if expiry_dropdown.value is not None:
        render_chain(index_dropdown.value, expiry_dropdown.value, preserve_selection=True)

btn_top3_ce.on_click(top3_ce_clicked)
btn_top3_pe.on_click(top3_pe_clicked)
btn_clear.on_click(clear_all_clicked)
btn_refresh.on_click(refresh_clicked)  # NEW

# =========================
# Dropdown handlers
# =========================
def on_index_change(change):
    if change['name'] == 'value':
        refresh_expiries(change['new'])
        if expiry_dropdown.value:
            render_chain(change['new'], expiry_dropdown.value)

def on_expiry_change(change):
    if change['name'] == 'value' and change['new'] is not None:
        render_chain(index_dropdown.value, change['new'])

index_dropdown.observe(on_index_change, names='value')
expiry_dropdown.observe(on_expiry_change, names='value')

# Initial load
refresh_expiries(index_dropdown.value)
if expiry_dropdown.value:
    render_chain(index_dropdown.value, expiry_dropdown.value)

# Helper used by Cell-3
def get_selected_symbols():
    return [sym for sym, cb in checkboxes.items() if cb.value]

**********************CONTROL LOSSES THAN MAKING PROFIT. MARKET WILL GIVE 2ND CHANCE****************************


Output()

In [ ]:
# ====== Cell 2 (manual settings) ======
# User-configurable parameters
stoploss_percent = 0.1              # 0.05 = 5% SL (change as per need)
trail_start_percent = 0.1          # Start trailing after 5% profit
trail_step_percent = 0.2            # Each trail step = 5% of buy price
square_off_target_percent = 2.0      # Fully exit trade after 20% profit

# Derived / extra config
enable_square_off = False            # Enable auto square-off when target achieved
log_folder = "/content/TradingLog"  # Colab working path where logs will be saved
# ===== New User Parameters =====
buffer_pct = 0.03                     # 2% safety-buffer while checking available funds
chunk_retry = 2                       # Retry attempts for chunked orders (SL / exit etc.)

In [ ]:
# @title
# ============================
# CELL 3 v2 — Fast + Round-Robin + Freeze-based max qty/order
# ============================

import os
import time
import signal
import sys
from datetime import datetime, date
from zoneinfo import ZoneInfo
from IPython.display import clear_output

# ----------------------------
# User confirmation + required vars
# ----------------------------
confirm = input("Every trade you take will take a toll on you health. So trade judiciously. You have to take only few CORRECT decisions, Is it one among them? (Y/N): ").strip().lower()
if confirm not in ("y", "yes"):
    raise SystemExit(" Aborted by user.")

required_vars = [
    "stoploss_percent",
    "trail_start_percent",
    "trail_step_percent",
    "square_off_target_percent",
    "log_folder",
]
missing = [v for v in required_vars if v not in globals() or globals()[v] is None]
if missing:
    raise NameError(f"Missing required vars: {missing}. Please define them before running Cell3 v2.")

# Optional settings with defaults
buffer_pct     = globals().get("buffer_pct", 0.02)         # safety buffer for fund usage
chunk_retry    = globals().get("chunk_retry", 2)           # retries for cancel/modify
exchange_prefix = globals().get("current_prefix", "NFO")   # set by selector cell
enable_square_off = globals().get("enable_square_off", True)

# Freeze limit per order (units), user can override globally
DEFAULT_MAX_FREEZE_QTY = globals().get("max_freeze_qty_per_order", 1800 if exchange_prefix == "NFO" else 600)

# Speed controls
FAST_MODE     = True
API_THROTTLE  = 0.08 if FAST_MODE else 0.60   # small delay between API calls
POLL_INTERVAL = 0.35 if FAST_MODE else 0.80   # how often to poll kite.orders()
FILL_TIMEOUT  = 12   if FAST_MODE else 30     # max seconds to wait for fills
MONITOR_SLEEP = 2.0  if FAST_MODE else 3.0    # monitor frequency

# ----------------------------
# Logging (IST timestamps)
# ----------------------------
os.makedirs(log_folder, exist_ok=True)
log_file_path = os.path.join(log_folder, date.today().isoformat() + ".txt")

def write_log(line: str):
    ts = datetime.now(ZoneInfo("Asia/Kolkata")).isoformat(timespec="seconds")
    with open(log_file_path, "a") as f:
        f.write(f"{ts}\n{line}\n")

write_log("=== New trading session started (Cell3 v2) ===")

# ----------------------------
# Kite wrappers
# ----------------------------
def kite_available_cash_once():
    try:
        m = kite.margins()
        return float(m["equity"]["available"]["live_balance"])
    except Exception as e:
        write_log(f"Could not fetch kite margins: {e}")
        return 0.0

def key_for(sym: str) -> str:
    return f"{exchange_prefix}:{sym}"

def get_ltps(symbols):
    """
    Batch LTP fetch: returns {sym: ltp}
    """
    if not symbols:
        return {}
    keys = [key_for(s) for s in symbols]
    try:
        data = kite.ltp(keys)
        out = {}
        for s in symbols:
            out[s] = data.get(key_for(s), {}).get("last_price")
        return out
    except Exception as e:
        write_log(f"get_ltps failed: {e}")
        return {s: None for s in symbols}

def place_market_buy(sym, qty):
    return kite.place_order(
        variety=kite.VARIETY_REGULAR,
        exchange=exchange_prefix,
        tradingsymbol=sym,
        transaction_type=kite.TRANSACTION_TYPE_BUY,
        quantity=int(qty),
        product=kite.PRODUCT_NRML,
        order_type=kite.ORDER_TYPE_MARKET
    )

def place_market_sell(sym, qty):
    return kite.place_order(
        variety=kite.VARIETY_REGULAR,
        exchange=exchange_prefix,
        tradingsymbol=sym,
        transaction_type=kite.TRANSACTION_TYPE_SELL,
        quantity=int(qty),
        product=kite.PRODUCT_NRML,
        order_type=kite.ORDER_TYPE_MARKET
    )

def place_sl_sell(sym, qty, sl_price):
    # NOTE: if your account/exchange requires trigger/limit gap, add a small gap here.
    return kite.place_order(
        variety=kite.VARIETY_REGULAR,
        exchange=exchange_prefix,
        tradingsymbol=sym,
        transaction_type=kite.TRANSACTION_TYPE_SELL,
        quantity=int(qty),
        product=kite.PRODUCT_NRML,
        order_type=kite.ORDER_TYPE_SL,
        price=float(sl_price),
        trigger_price=float(sl_price)
    )

# ----------------------------
# Orders polling (batch, fast)
# ----------------------------
TERMINAL = {"COMPLETE", "REJECTED", "CANCELLED"}

def fetch_orders_map():
    try:
        all_orders = kite.orders()
        return {o.get("order_id"): o for o in all_orders if o.get("order_id")}
    except Exception as e:
        write_log(f"fetch_orders_map failed: {e}")
        return {}

def wait_for_orders(order_ids, timeout_sec=12, poll_interval=0.35):
    order_ids = [oid for oid in order_ids if oid]
    if not order_ids:
        return {}

    start = time.time()
    last_seen = {}

    while time.time() - start < timeout_sec:
        omap = fetch_orders_map()
        for oid in order_ids:
            if oid in omap:
                last_seen[oid] = omap[oid]

        done = sum(1 for oid in order_ids if (last_seen.get(oid, {}) or {}).get("status") in TERMINAL)
        if done == len(order_ids):
            return last_seen

        time.sleep(poll_interval)

    write_log(f"wait_for_orders timeout after {timeout_sec}s; returning partial statuses.")
    return last_seen

# ----------------------------
# Lot size: prefer instrument lookup, fallback to heuristic
# ----------------------------
_lot_size_cache = {}

def build_lot_size_cache(selected_syms):
    """
    Build a dict {tradingsymbol: lot_size} by scanning kite.instruments(segment) once.
    Falls back if not found.
    """
    global _lot_size_cache
    try:
        inst = kite.instruments(exchange_prefix)  # segment is NFO/BFO etc.
        need = set(selected_syms)
        cache = {}
        for i in inst:
            ts = i.get("tradingsymbol")
            if ts in need:
                ls = i.get("lot_size") or i.get("market_lot") or i.get("lot_size")
                if ls:
                    cache[ts] = int(ls)
        _lot_size_cache = cache
        write_log(f"Lot size cache built for {len(cache)}/{len(selected_syms)} symbols.")
    except Exception as e:
        write_log(f"Lot size cache build failed: {e}")
        _lot_size_cache = {}

def infer_lot_size(sym: str) -> int:
    """
    Use cached instrument lot size if available.
    Else fallback (NIFTY=75, SENSEX=20) heuristic.
    """
    if sym in _lot_size_cache:
        return int(_lot_size_cache[sym])

    # Fallback heuristic (kept for compatibility)
    if "SENSEX" in sym.upper() or exchange_prefix == "BFO":
        return 20
    return 75

# ----------------------------
# Chunking utilities (freeze-based max qty/order)
# ----------------------------
def max_units_per_order(lot_size: int) -> int:
    """
    Max units per single order based on freeze limit, aligned to lot size.
    """
    max_units = int(DEFAULT_MAX_FREEZE_QTY)
    lots = max(1, max_units // lot_size)
    return int(lots * lot_size)

def aligned_affordable_chunk(remaining_funds, ltp, lot_size, max_units, buffer_pct):
    """
    Largest chunk aligned to lot size, capped by freeze-based max_units.
    """
    if not ltp or ltp <= 0:
        return 0
    affordable_units = int(remaining_funds // (ltp * (1 + buffer_pct)))
    affordable_lots = affordable_units // lot_size
    if affordable_lots <= 0:
        return 0
    units = affordable_lots * lot_size
    return int(min(units, max_units))

def split_units_to_chunks(total_units, max_units, lot_size):
    """
    If you need chunk splitting (sells / SL), align each chunk to lot size.
    """
    chunks = []
    remaining = int(total_units)
    if max_units < lot_size:
        max_units = lot_size

    while remaining > 0:
        take = min(remaining, max_units)
        rem_mod = take % lot_size
        if rem_mod != 0:
            take -= rem_mod
            if take <= 0:
                take = lot_size
        chunks.append(int(take))
        remaining -= take
    return chunks

# ----------------------------
# Registries
# ----------------------------
open_positions_registry = {}  # {sym: {"qty": int, "sl_order_ids":[...]}}
positions = {}                # {sym: state dict}

# ----------------------------
# SL helpers (split SL)
# ----------------------------
def place_split_sl_orders(sym, total_units, sl_price, lot_size):
    max_u = max_units_per_order(lot_size)
    chunks = split_units_to_chunks(total_units, max_u, lot_size)
    placed = []
    for chunk in chunks:
        try:
            oid = place_sl_sell(sym, chunk, sl_price)
            placed.append({"order_id": oid, "qty": chunk, "price": float(sl_price)})
            time.sleep(API_THROTTLE)
        except Exception as e:
            write_log(f"ERROR placing SL chunk {sym} qty={chunk}: {e}")
    return placed

def cancel_order_fast(order_id):
    try:
        kite.cancel_order(variety=kite.VARIETY_REGULAR, order_id=order_id)
        return True
    except Exception as e:
        # ignore common states (already complete/cancelled)
        write_log(f"cancel_order_fast ignore {order_id}: {e}")
        return False

def cancel_sl_fast(order_ids):
    for oid in order_ids or []:
        if oid:
            cancel_order_fast(oid)
            time.sleep(API_THROTTLE)

def modify_sl_order_with_retry(order_id, new_price, symbol):
    for attempt in range(1, chunk_retry + 1):
        try:
            kite.modify_order(
                variety=kite.VARIETY_REGULAR,
                order_id=order_id,
                price=float(new_price),
                trigger_price=float(new_price)
            )
            return True
        except Exception as e:
            write_log(f"{symbol} ERROR modify SL {order_id} -> {new_price} attempt={attempt}: {e}")
            time.sleep(0.25 * attempt if FAST_MODE else 0.8 * (2 ** (attempt - 1)))
    return False

# ----------------------------
# Fast safe_exit: cancel SLs then market sell in freeze-aligned chunks
# ----------------------------
def split_market_sell(sym, total_units, lot_size):
    max_u = max_units_per_order(lot_size)
    chunks = split_units_to_chunks(total_units, max_u, lot_size)
    sell_oids = []
    for chunk in chunks:
        try:
            oid = place_market_sell(sym, chunk)
            sell_oids.append(oid)
            time.sleep(API_THROTTLE)
        except Exception as e:
            write_log(f"ERROR market sell {sym} chunk={chunk}: {e}")
    # optional: wait briefly for sells to complete
    wait_for_orders(sell_oids, timeout_sec=FILL_TIMEOUT, poll_interval=POLL_INTERVAL)
    return sell_oids

def safe_exit(sym, reason="manual"):
    st = positions.get(sym, {}) or {}
    qty = int(st.get("quantity", 0) or open_positions_registry.get(sym, {}).get("qty", 0) or 0)
    if qty <= 0:
        return

    write_log(f"[safe_exit] {sym} qty={qty} reason={reason}")

    # Cancel all SL orders first to prevent accidental short
    sl_ids = []
    sl_ids.extend(open_positions_registry.get(sym, {}).get("sl_order_ids", []) or [])
    sl_ids.extend([e.get("order_id") for e in (st.get("sl_orders") or []) if e.get("order_id")])
    cancel_sl_fast(list(set(sl_ids)))

    # Sell in freeze-aligned chunks
    lot_size = int(st.get("lot_size") or infer_lot_size(sym))
    split_market_sell(sym, qty, lot_size)

    # Cleanup
    open_positions_registry.pop(sym, None)
    positions.pop(sym, None)
    write_log(f"[safe_exit] Completed cleanup for {sym}")

# ----------------------------
# Emergency handler: FAST exit all
# ----------------------------
def emergency_exit_handler(signum, frame):
    print("\n EMERGENCY STOP — FAST EXIT ALL...")
    write_log("EMERGENCY STOP received. FAST EXIT ALL started.")

    # Cancel all SL ids across all symbols first
    all_sl = set()
    for sym, reg in list(open_positions_registry.items()):
        for oid in (reg.get("sl_order_ids") or []):
            if oid:
                all_sl.add(oid)
    for sym, st in list(positions.items()):
        for e in (st.get("sl_orders") or []):
            oid = e.get("order_id")
            if oid:
                all_sl.add(oid)

    cancel_sl_fast(list(all_sl))

    # Then sell all
    for sym in list(open_positions_registry.keys()):
        try:
            safe_exit(sym, reason="emergency")
        except Exception as e:
            write_log(f"Emergency safe_exit failed for {sym}: {e}")

    write_log("EMERGENCY EXIT COMPLETE")
    print(" Emergency exit finished. Exiting notebook.")
    sys.exit(0)

signal.signal(signal.SIGINT, emergency_exit_handler)

# ----------------------------
# Selection validation (from selector cell)
# ----------------------------
selected = get_selected_symbols()
if not selected:
    raise SystemExit(" No symbols selected. Select in chain UI and re-run.")

if "symbol_meta" not in globals() or not symbol_meta:
    raise SystemExit(" symbol_meta missing. Re-render selector chain and re-select.")

first_meta = symbol_meta[selected[0]]
index_name = first_meta.get("index_name")

atm_data = atm_info.get(index_name, {}) if "atm_info" in globals() else {}
atm_strike = atm_data.get("atm_strike", None)

def strike_distance(sym):
    if atm_strike is None:
        return 0
    return abs((symbol_meta.get(sym, {}) or {}).get("strike", 10**18) - atm_strike)

selected_sorted = sorted(selected, key=strike_distance)
write_log(f"Selected symbols (sorted): {selected_sorted}")

print("Selected symbols:")
for s in selected_sorted:
    m = symbol_meta.get(s, {}) or {}
    print(f" - {s} strike={m.get('strike')} type={m.get('instrument_type')}")

# Build lot size cache once (fast lookups later)
build_lot_size_cache(selected_sorted)

# ----------------------------
# Funds input
# ----------------------------
initial_cash = kite_available_cash_once()
print(f"Available Live Balance (once) = ₹{initial_cash:.2f}")
user_input = input(f"Enter fund to use (press Enter to use {initial_cash:.2f}): ").strip()
fund = float(user_input) if user_input != "" else initial_cash
write_log(f"Fund allocated: ₹{fund:.2f}")
print(f"Fund set to ₹{fund:.2f}")

remaining_funds = fund * (1 - buffer_pct)
write_log(f"Usable funds after buffer ({buffer_pct*100:.1f}%): ₹{remaining_funds:.2f}")

# ----------------------------
# ROUND-ROBIN ALLOCATION
# ----------------------------
print("\nStarting ROUND-ROBIN allocation (freeze-based max qty/order)...")
write_log("Starting ROUND-ROBIN allocation...")

placed_orders = []  # list of {sym, oid, req_qty, est_ltp}
progress = True

# Pre-compute per symbol constraints
lot_map = {s: infer_lot_size(s) for s in selected_sorted}
maxu_map = {s: max_units_per_order(lot_map[s]) for s in selected_sorted}

# ltp cache for estimation
ltp_cache = {}

while remaining_funds > 0 and progress:
    progress = False

    # One round: each symbol gets at most one order
    ltps = get_ltps(selected_sorted)  # batch
    for sym in selected_sorted:
        ltp = ltps.get(sym) or ltp_cache.get(sym)
        if not ltp:
            continue
        ltp_cache[sym] = ltp

        lot_size = lot_map[sym]
        max_units = maxu_map[sym]

        chunk = aligned_affordable_chunk(remaining_funds, ltp, lot_size, max_units, buffer_pct)
        if chunk <= 0:
            continue

        try:
            oid = place_market_buy(sym, chunk)
            placed_orders.append({"sym": sym, "order_id": oid, "req_qty": int(chunk), "est_ltp": float(ltp)})
            write_log(f"RR BUY placed {sym} qty={chunk} oid={oid} est_ltp={ltp}")

            # reserve conservatively using ltp
            remaining_funds = max(0.0, remaining_funds - (ltp * chunk))
            progress = True
            time.sleep(API_THROTTLE)
        except Exception as e:
            write_log(f"RR BUY failed {sym} qty={chunk}: {e}")

if not placed_orders:
    write_log("No purchases made. Exiting.")
    raise SystemExit(" No purchases were made with provided funds / available LTPs. Exiting.")

# Batch wait for fills
order_ids = [x["order_id"] for x in placed_orders]
status_map = wait_for_orders(order_ids, timeout_sec=FILL_TIMEOUT, poll_interval=POLL_INTERVAL)

# Aggregate fills per symbol
fills_by_sym = {}
for rec in placed_orders:
    sym = rec["sym"]
    oid = rec["order_id"]
    od = status_map.get(oid, {}) or {}
    st = od.get("status")

    if st == "REJECTED":
        write_log(f"Order REJECTED: {sym} oid={oid}")
        continue

    filled_qty = int(od.get("filled_quantity") or od.get("quantity") or rec["req_qty"])
    avg_px = float(od.get("average_price") or rec["est_ltp"] or 0.0)
    if filled_qty <= 0:
        continue

    agg = fills_by_sym.get(sym, {"qty": 0, "w_sum": 0.0, "orders": []})
    agg["qty"] += filled_qty
    agg["w_sum"] += avg_px * filled_qty
    agg["orders"].append({"order_id": oid, "qty": filled_qty, "avg_price": avg_px, "status": st})
    fills_by_sym[sym] = agg

# Place SL per symbol after aggregation
for sym, agg in fills_by_sym.items():
    qty = agg["qty"]
    avg_buy = agg["w_sum"] / qty if qty > 0 else (ltp_cache.get(sym) or 0.0)

    lot_size = lot_map[sym]
    current_sl = round(avg_buy * (1 - stoploss_percent), 1)

    sl_entries = []
    try:
        sl_entries = place_split_sl_orders(sym, qty, current_sl, lot_size)
        write_log(f"SL placed for {sym} chunks={len(sl_entries)} at {current_sl}")
    except Exception as e:
        write_log(f"ERROR placing SL for {sym}: {e}")
        print(f" SL placement failed for {sym}: {e}")

    sl_ids = [e.get("order_id") for e in sl_entries if e.get("order_id")]
    open_positions_registry[sym] = {"qty": int(qty), "sl_order_ids": sl_ids}

    positions[sym] = {
        "quantity": int(qty),
        "avg_buy_price": float(avg_buy),
        "sl_orders": sl_entries,
        "current_sl": float(current_sl),
        "lot_size": int(lot_size),
        "sub_orders": agg["orders"],
    }

    print(f"Bought {sym} qty={qty} avg={avg_buy:.2f} SL={current_sl}")
    write_log(f"AGG BUY {sym} qty={qty} avg={avg_buy} SL={current_sl} sl_chunks={len(sl_entries)}")

if not positions:
    write_log("No positions to monitor after allocation. Exiting.")
    raise SystemExit(" No positions created. Exiting.")

write_log(f"Allocation complete. Remaining usable funds (est) ₹{remaining_funds:.2f}")
print("\nAllocation complete. Entering monitoring phase.")
print("Monitoring... Press Ctrl+C for EMERGENCY STOP (fast exit).")

# ----------------------------
# Monitoring loop (faster + safer)
# ----------------------------
log_buffer = []
try:
    while True:
        if not positions:
            write_log("All positions closed. Exiting monitor.")
            break

        # Fetch kite positions ONCE per cycle
        pos_data = {}
        try:
            p = kite.positions()
            pos_data = {(x.get("exchange"), x.get("tradingsymbol")): x.get("quantity", 0)
                        for x in (p.get("net", []) or [])}
        except Exception as e:
            write_log(f"positions() fetch failed: {e}")
            pos_data = {}

        syms = list(positions.keys())
        ltps = get_ltps(syms)  # batch LTPs

        # Orders map once per cycle (for SL status checks)
        orders_map = fetch_orders_map()

        # Display area
        clear_output(wait=True)
        print("=== Trading monitor (Cell3 v2) ===")
        print(f"Remaining usable funds (est): ₹{remaining_funds:.2f}")
        print("Tracked symbols:", syms)
        print("-" * 50)

        # Square-off trigger flag
        square_off_now = False

        for sym in list(positions.keys()):
            st = positions.get(sym, {}) or {}
            qty_open = pos_data.get((exchange_prefix, sym), None)

            # If broker reports closed, stop tracking
            if qty_open is not None and qty_open <= 0:
                write_log(f"{sym} closed (broker qty=0). Removing from tracking.")
                positions.pop(sym, None)
                open_positions_registry.pop(sym, None)
                continue

            ltp = ltps.get(sym)
            if not ltp:
                continue

            avg_buy = st["avg_buy_price"]
            profit_pct = (ltp - avg_buy) / avg_buy if avg_buy else 0.0

            msg = f"{sym} LTP:{ltp:.2f} Pct:{profit_pct*100:.2f}% SL:{st['current_sl']}"
            log_buffer.append(msg)
            log_buffer = log_buffer[-20:]
            print(msg)

            # --- SL miss protection (improved for multi-SL chunks) ---
            sl_entries = st.get("sl_orders", []) or []
            sl_ids = [e.get("order_id") for e in sl_entries if e.get("order_id")]

            if sl_ids and ltp <= st["current_sl"]:
                # Determine if *any* SL is actually in terminal COMPLETE
                statuses = []
                for oid in sl_ids:
                    od = orders_map.get(oid, {}) or {}
                    statuses.append(od.get("status"))

                any_complete = any(s == "COMPLETE" for s in statuses if s)
                any_openish = any(s in ("OPEN", "TRIGGER PENDING", None) for s in statuses)

                # If price crossed SL and nothing completed, force exit fast
                if (not any_complete) and any_openish:
                    write_log(f"SL miss detected for {sym} (ltp<=SL). Forcing safe_exit.")
                    safe_exit(sym, reason="sl_miss_force_exit")
                    continue

            # --- Trailing SL ---
            if profit_pct >= trail_start_percent:
                trail_step_value = avg_buy * trail_step_percent
                new_sl = round(ltp - trail_step_value, 1)
                if new_sl > st["current_sl"]:
                    # modify all SL orders
                    ok_count = 0
                    for e in sl_entries:
                        oid = e.get("order_id")
                        if not oid:
                            continue
                        if modify_sl_order_with_retry(oid, new_sl, sym):
                            ok_count += 1
                            e["price"] = float(new_sl)
                        time.sleep(API_THROTTLE)
                    st["current_sl"] = float(new_sl)
                    write_log(f"Trailing SL updated {sym} -> {new_sl} (modified {ok_count}/{len(sl_entries)})")

            # --- Square-off target ---
            if enable_square_off and profit_pct >= square_off_target_percent:
                square_off_now = True

        # Global square-off: exit all open positions
        if enable_square_off and square_off_now:
            write_log("Square-off target reached for at least one symbol. Exiting all positions.")
            print("\n Square-off target reached. Exiting ALL positions...")
            for sym in list(positions.keys()):
                safe_exit(sym, reason="square_off")
            continue

        time.sleep(MONITOR_SLEEP)

except KeyboardInterrupt:
    write_log("KeyboardInterrupt caught; triggering emergency exit handler.")
    emergency_exit_handler(None, None)
finally:
    write_log(" Strategy stopped safely (Cell3 v2).")
    print(" Strategy stopped safely (Cell3 v2).")

=== Trading monitor (Cell3 v2) ===
Remaining usable funds (est): ₹3475.51
Tracked symbols: ['NIFTY2621725600CE', 'NIFTY2621725650CE']
--------------------------------------------------
NIFTY2621725600CE LTP:100.05 Pct:4.83% SL:85.9
NIFTY2621725650CE LTP:77.20 Pct:4.75% SL:66.3

 EMERGENCY STOP — FAST EXIT ALL...
 Emergency exit finished. Exiting notebook.
 Strategy stopped safely (Cell3 v2).


SystemExit: 0

In [ ]:
# @title
# # @title
# # ============================
# # CELL 3 - Multi-symbol entry, split SL, trailing, safe_exit, per-day IST logging
# # Paste into Google Colab and run (requires kite, symbol_meta, atm_info, get_selected_symbols())
# # ============================

# import os
# import time
# import signal
# import sys
# from datetime import datetime, date
# from zoneinfo import ZoneInfo
# from IPython.display import clear_output

# # ----------------------------
# # Basic confirmation + required globals check
# # ----------------------------
# confirm = input('''You have to take only few CORRECT decisions, Is it among them? (Y/N): ''').strip().lower()
# if confirm not in ("y", "yes"):
#     raise SystemExit("❌ Aborted by user.")

# required_vars = [
#     "stoploss_percent",
#     "trail_start_percent",
#     "trail_step_percent",
#     "square_off_target_percent",
#     "log_folder",
#     # buffer_pct and chunk_retry are optional but will be read if present
# ]
# missing = [v for v in required_vars if v not in globals() or globals()[v] is None]
# if missing:
#     raise NameError(f"Missing required vars: {missing}. Please define them in Cell-2.")

# # Read optional settings from Cell-2 (with defaults)
# buffer_pct = globals().get("buffer_pct", 0.02)     # safety buffer when estimating required cash
# chunk_retry = globals().get("chunk_retry", 3)      # attempts for retries on SL ops and chunk buys
# exchange_prefix = globals().get("current_prefix", "NFO")

# # ----------------------------
# # Logging helper (IST timestamps)
# # ----------------------------
# os.makedirs(log_folder, exist_ok=True)
# log_file_path = os.path.join(log_folder, date.today().isoformat() + ".txt")

# def write_log(line):
#     ts = datetime.now(ZoneInfo("Asia/Kolkata")).isoformat(timespec="seconds")
#     with open(log_file_path, "a") as f:
#         f.write(f"{ts} | {line}\n")

# write_log("=== New trading session started ===")

# # ----------------------------
# # Global registries
# # ----------------------------
# # open_positions_registry: { sym: {"qty": int, "exchange": "NFO", "sl_order_ids": [...]} }
# open_positions_registry = {}
# # positions: aggregated per-run positions with sl chunks etc.
# positions = {}

# # ----------------------------
# # Kite wrappers
# # ----------------------------
# def kite_available_cash_once():
#     try:
#         m = kite.margins()
#         return float(m['equity']['available']['live_balance'])
#     except Exception as e:
#         write_log(f"Could not fetch kite margins (once): {e}")
#         return 0.0

# def get_ltp(sym):
#     key = f"{exchange_prefix}:{sym}"
#     data = kite.ltp([key])
#     return data[key]["last_price"]

# def place_market_buy(sym, qty):
#     return kite.place_order(
#         variety=kite.VARIETY_REGULAR,
#         exchange=exchange_prefix,
#         tradingsymbol=sym,
#         transaction_type=kite.TRANSACTION_TYPE_BUY,
#         quantity=qty,
#         product=kite.PRODUCT_NRML,
#         order_type=kite.ORDER_TYPE_MARKET
#     )

# def place_market_sell(sym, qty):
#     return kite.place_order(
#         variety=kite.VARIETY_REGULAR,
#         exchange=exchange_prefix,
#         tradingsymbol=sym,
#         transaction_type=kite.TRANSACTION_TYPE_SELL,
#         quantity=qty,
#         product=kite.PRODUCT_NRML,
#         order_type=kite.ORDER_TYPE_MARKET
#     )

# def place_sl_sell_single(sym, qty, sl_price):
#     return kite.place_order(
#         variety=kite.VARIETY_REGULAR,
#         exchange=exchange_prefix,
#         tradingsymbol=sym,
#         transaction_type=kite.TRANSACTION_TYPE_SELL,
#         quantity=qty,
#         product=kite.PRODUCT_NRML,
#         order_type=kite.ORDER_TYPE_SL,
#         price=sl_price,
#         trigger_price=sl_price
#     )

# def cancel_order_safe(order_id):
#     try:
#         kite.cancel_order(variety=kite.VARIETY_REGULAR, order_id=order_id)
#         return True
#     except Exception as e:
#         write_log(f"cancel_order_safe failed for {order_id}: {e}")
#         return False

# # ----------------------------
# # Utilities
# # ----------------------------
# def infer_lot_size(sym):
#     # adjust if BFO or SENSEX
#     if "SENSEX" in sym.upper() or exchange_prefix == "BFO":
#         return 20
#     return 75

# # Default freeze qty: allow override via global 'max_freeze_qty_per_order'
# if 'max_freeze_qty_per_order' in globals() and isinstance(globals()['max_freeze_qty_per_order'], int):
#     DEFAULT_MAX_FREEZE_QTY = globals()['max_freeze_qty_per_order']
# else:
#     DEFAULT_MAX_FREEZE_QTY = 1800 if exchange_prefix == "NFO" else 600

# def split_units_to_chunks(total_units, max_units_per_order, lot_size):
#     chunks = []
#     remaining = int(total_units)
#     if max_units_per_order < lot_size:
#         max_units_per_order = lot_size
#     while remaining > 0:
#         take = min(remaining, max_units_per_order)
#         rem_mod = take % lot_size
#         if rem_mod != 0:
#             adj = take - rem_mod
#             if adj <= 0:
#                 take = lot_size
#             else:
#                 take = adj
#         chunks.append(int(take))
#         remaining -= take
#     return chunks

# # ----------------------------
# # Sell helper (split aware) — updates registry
# # ----------------------------
# def split_market_sell_and_update(sym, total_units, lot_size, max_units_per_order=None):
#     executed = []
#     if max_units_per_order is None:
#         max_units_per_order = DEFAULT_MAX_FREEZE_QTY
#     chunks = split_units_to_chunks(total_units, max_units_per_order, lot_size)
#     for chunk in chunks:
#         try:
#             write_log(f"Placing MARKET SELL chunk for {sym} units={chunk}")
#             oid = place_market_sell(sym, chunk)
#             # small wait to avoid hammering Kite
#             time.sleep(0.6)
#             try:
#                 hist = kite.order_history(oid)
#                 rec = hist[-1]
#                 avg_px = rec.get("average_price") or get_ltp(sym)
#                 filled_qty = rec.get("filled_quantity") or rec.get("quantity") or chunk
#             except Exception:
#                 avg_px = get_ltp(sym)
#                 filled_qty = chunk
#             executed.append({"order_id": oid, "qty": int(filled_qty), "avg_price": avg_px})
#             write_log(f"Chunk SELL executed {sym} qty={filled_qty} avg_price={avg_px} order_id={oid}")
#             # update registry
#             prev = open_positions_registry.get(sym, {"qty": 0, "exchange": exchange_prefix, "sl_order_ids": []})
#             prev_qty = prev.get("qty", 0)
#             new_qty = max(0, prev_qty - int(filled_qty))
#             if new_qty > 0:
#                 slid_list = prev.get("sl_order_ids", [])
#                 open_positions_registry[sym] = {"qty": new_qty, "exchange": exchange_prefix, "sl_order_ids": slid_list}
#             else:
#                 open_positions_registry.pop(sym, None)
#         except Exception as e:
#             write_log(f"ERROR placing chunk SELL for {sym} units={chunk}: {e}")
#     return executed

# # ----------------------------
# # Place split stoploss orders (matching aggregated qty)
# # ----------------------------
# def place_split_sl_orders(sym, total_units, sl_price, lot_size, max_units_per_order=None):
#     if max_units_per_order is None:
#         max_units_per_order = DEFAULT_MAX_FREEZE_QTY
#     chunks = split_units_to_chunks(total_units, max_units_per_order, lot_size)
#     placed = []
#     for chunk in chunks:
#         try:
#             write_log(f"Placing SL chunk for {sym} qty={chunk} sl={sl_price}")
#             oid = place_sl_sell_single(sym, chunk, sl_price)
#             time.sleep(0.4)
#             placed.append({"order_id": oid, "qty": chunk, "price": sl_price})
#             write_log(f"Placed SL chunk {sym} qty={chunk} order_id={oid}")
#         except Exception as e:
#             write_log(f"ERROR placing SL chunk for {sym} qty={chunk}: {e}")
#     return placed

# # ----------------------------
# # Retry + exponential/backoff helpers for SL modify & cancel
# # ----------------------------
# def cancel_sl_order_with_retry(order_id, symbol):
#     for attempt in range(1, chunk_retry + 1):
#         try:
#             kite.cancel_order(variety=kite.VARIETY_REGULAR, order_id=order_id)
#             write_log(f"{symbol} | SL cancel success | order_id={order_id} | attempt={attempt}")
#             return True
#         except Exception as e:
#             write_log(f"{symbol} | ERROR cancelling SL order {order_id} (attempt={attempt}): {e}")
#             time.sleep(0.6 * (2 ** (attempt-1)))
#     write_log(f"{symbol} | FAILED cancelling SL {order_id} after {chunk_retry} attempts")
#     return False

# def modify_sl_order_with_retry(order_id, new_price, symbol):
#     for attempt in range(1, chunk_retry + 1):
#         try:
#             kite.modify_order(variety=kite.VARIETY_REGULAR, order_id=order_id, price=new_price, trigger_price=new_price)
#             write_log(f"{symbol} | SL modified -> {new_price} | order_id={order_id} | attempt={attempt}")
#             return True
#         except Exception as e:
#             write_log(f"{symbol} | ERROR modifying SL {order_id} -> {new_price} (attempt={attempt}): {e}")
#             time.sleep(0.8 * (2 ** (attempt-1)))
#     write_log(f"{symbol} | FAILED modifying SL {order_id} after {chunk_retry} attempts")
#     return False

# def cancel_sl_list(sl_order_ids):
#     if not sl_order_ids:
#         return
#     for oid in sl_order_ids:
#         if oid:
#             cancel_sl_order_with_retry(oid, "SL")

# def modify_sl_list(sl_order_entries, new_price, symbol):
#     results = []
#     for entry in sl_order_entries or []:
#         oid = entry.get("order_id")
#         if oid:
#             ok = modify_sl_order_with_retry(oid, new_price, symbol)
#             results.append((oid, ok))
#             if ok:
#                 entry["price"] = new_price
#     return results

# # ----------------------------
# # Unified safe_exit() - cancel SLs then split sell and cleanup
# # ----------------------------
# def safe_exit(symbol, qty, reason="manual"):
#     write_log(f"[safe_exit] Initiating exit for {symbol} qty={qty} reason={reason}")
#     print(f"⚠️ safe_exit: Exiting {symbol} qty={qty} (reason={reason})")
#     reg = open_positions_registry.get(symbol, {}) or {}
#     sl_ids = reg.get("sl_order_ids", []) or []
#     pos = positions.get(symbol, {}) or {}
#     pos_sl_entries = pos.get("sl_orders", []) or []
#     try:
#         if sl_ids:
#             cancel_sl_list(sl_ids)
#         if pos_sl_entries:
#             cancel_sl_list([e.get("order_id") for e in pos_sl_entries if e.get("order_id")])
#     except Exception as e:
#         write_log(f"[safe_exit] Error cancelling SLs for {symbol}: {e}")
#     lot_size = infer_lot_size(symbol)
#     executed_chunks = []
#     try:
#         executed_chunks = split_market_sell_and_update(symbol, qty, lot_size, DEFAULT_MAX_FREEZE_QTY)
#         write_log(f"[safe_exit] SELL completed for {symbol}, chunks={len(executed_chunks)}")
#         print(f"✔ safe_exit: SELL executed for {symbol}")
#     except Exception as e:
#         write_log(f"[safe_exit] ERROR SELL for {symbol}: {e}")
#         print(f"❌ safe_exit SELL failed for {symbol}: {e}")
#     try:
#         open_positions_registry.pop(symbol, None)
#         positions.pop(symbol, None)
#         write_log(f"[safe_exit] Registry and positions cleaned for {symbol}")
#     except Exception as e:
#         write_log(f"[safe_exit] ERROR cleaning registry for {symbol}: {e}")
#     return executed_chunks

# # ----------------------------
# # Emergency handler (SIGINT) -> uses safe_exit
# # ----------------------------
# def emergency_exit_handler(signum, frame):
#     print("\n🛑 EMERGENCY STOP RECEIVED — Exiting all open positions now...")
#     write_log("EMERGENCY STOP received. Attempting to exit all positions.")
#     for sym, p in list(open_positions_registry.items()):
#         qty = p.get("qty", 0)
#         if qty > 0:
#             try:
#                 safe_exit(sym, qty, reason="emergency")
#                 write_log(f"Emergency exit executed for {sym}")
#                 print(f"✔ Emergency exit executed for {sym} qty={qty}")
#             except Exception as e:
#                 write_log(f"ERROR emergency exiting {sym}: {e}")
#                 print(f"❌ Emergency exit failed for {sym}: {e}")
#     write_log("EMERGENCY EXIT COMPLETE")
#     print("🔒 Emergency exit routine finished. Exiting notebook.")
#     sys.exit(0)

# signal.signal(signal.SIGINT, emergency_exit_handler)

# # ----------------------------
# # Selection & validation
# # ----------------------------
# selected = get_selected_symbols()
# if not selected:
#     raise SystemExit("⚠️ No symbols selected in Cell 1. Select and re-run.")
# if "symbol_meta" not in globals() or not symbol_meta:
#     raise SystemExit("⚠️ symbol_meta missing. Re-render Cell 1 chain and re-select.")

# first_meta = symbol_meta[selected[0]]
# index_name = first_meta.get("index_name")
# atm_data = atm_info.get(index_name, {}) if "atm_info" in globals() else {}
# atm_strike = atm_data.get("atm_strike", None)
# def strike_distance(sym):
#     return abs(symbol_meta.get(sym, {}).get("strike", 1e9) - (atm_strike or 0)) if atm_strike is not None else 0
# selected_sorted = sorted(selected, key=lambda s: strike_distance(s))

# write_log(f"Selected symbols: {selected_sorted}")
# print("Selected symbols (closest-to-ATM first if available):")
# for s in selected_sorted:
#     meta = symbol_meta.get(s, {})
#     print(f" - {s} | strike={meta.get('strike')} | type={meta.get('instrument_type')}")

# # ----------------------------
# # Margins once + fund input
# # ----------------------------
# initial_available_cash = kite_available_cash_once()
# print(f"Available Live Balance (once) = ₹{initial_available_cash:.2f}")
# user_input = input(f"Enter fund to use (press Enter to use {initial_available_cash:.2f}): ").strip()
# fund = float(user_input) if user_input != "" else initial_available_cash
# write_log(f"Fund allocated: ₹{fund:.2f}")
# print(f"Fund set to ₹{fund:.2f}")

# # usable pool after buffer
# remaining_funds = fund * (1 - buffer_pct)
# write_log(f"Usable funds after buffer ({buffer_pct*100:.1f}%): ₹{remaining_funds:.2f}")

# # prefetch ltps
# ltp_cache = {}
# for sym in selected_sorted:
#     try:
#         ltp_cache[sym] = get_ltp(sym)
#     except Exception as e:
#         ltp_cache[sym] = None
#         write_log(f"Could not fetch LTP for {sym}: {e}")

# write_log("Starting allocation.")
# print("\nStarting allocation...")

# # ----------------------------
# # Allocation: iterate symbols and buy with adaptive chunking (faster: uses remaining_funds cache)
# # ----------------------------
# any_buys = False

# for sym in selected_sorted:
#     if remaining_funds <= 0:
#         write_log("Remaining funds exhausted, stopping allocation.")
#         break

#     try:
#         ltp = get_ltp(sym)
#         ltp_cache[sym] = ltp
#     except Exception as e:
#         write_log(f"Skipping buy attempt for {sym} due to LTP error: {e}")
#         continue

#     lot_size = infer_lot_size(sym)
#     max_units_affordable = int(remaining_funds // ltp) if ltp and ltp > 0 else 0
#     max_lots_affordable = max_units_affordable // lot_size
#     if max_lots_affordable <= 0:
#         write_log(f"Not enough usable funds for even 1 lot of {sym}. skip.")
#         continue

#     max_freeze_units = globals().get('max_freeze_qty_per_order', DEFAULT_MAX_FREEZE_QTY)
#     max_lots_per_order = max(1, max_freeze_units // lot_size)
#     max_units_per_order = max_lots_per_order * lot_size

#     desired_units = max_lots_affordable * lot_size
#     chunks = split_units_to_chunks(desired_units, max_units_per_order, lot_size)
#     if not chunks:
#         continue

#     executed_total = 0
#     weighted_sum = 0.0
#     sub_order_records = []

#     for base_chunk in chunks:
#         chunk = base_chunk
#         # try chunk; on failure reduce by one lot and retry
#         while chunk >= lot_size:
#             try:
#                 cur_ltp = get_ltp(sym)
#             except Exception:
#                 cur_ltp = ltp_cache.get(sym) or ltp
#             est_required = cur_ltp * chunk * (1 + buffer_pct)
#             if remaining_funds >= est_required:
#                 try:
#                     write_log(f"Placing MARKET BUY chunk for {sym} units={chunk} est_cost=₹{est_required:.2f}")
#                     oid = place_market_buy(sym, chunk)
#                     time.sleep(0.8)
#                     try:
#                         hist = kite.order_history(oid)
#                         rec = hist[-1]
#                         avg_px = rec.get("average_price") or cur_ltp
#                         filled_qty = rec.get("filled_quantity") or rec.get("quantity") or chunk
#                     except Exception:
#                         avg_px = cur_ltp
#                         filled_qty = chunk
#                         write_log(f"WARNING: couldn't get order_history for {oid} of {sym}")
#                     executed_total += int(filled_qty)
#                     weighted_sum += (avg_px * filled_qty)
#                     sub_order_records.append({"order_id": oid, "qty": int(filled_qty), "avg_price": avg_px})
#                     any_buys = True
#                     used_cash = avg_px * filled_qty
#                     remaining_funds = max(0.0, remaining_funds - used_cash)
#                     prev = open_positions_registry.get(sym, {"qty": 0, "exchange": exchange_prefix, "sl_order_ids": []})
#                     open_positions_registry[sym] = {"qty": prev.get("qty", 0) + int(filled_qty), "exchange": exchange_prefix, "sl_order_ids": prev.get("sl_order_ids", [])}
#                     write_log(f"Chunk BUY executed {sym} qty={filled_qty} avg_price={avg_px} order_id={oid} remaining_usable=₹{remaining_funds:.2f}")
#                     print(f"Chunk BUY {sym} qty={filled_qty} avg={avg_px} -> Remaining usable ₹{remaining_funds:.2f}")
#                     break
#                 except Exception as e:
#                     write_log(f"BUY chunk failed for {sym} units={chunk}: {e}")
#                     chunk -= lot_size
#                     write_log(f"Reducing chunk for {sym} to {chunk} and retrying.")
#                     time.sleep(0.2)
#                     continue
#             else:
#                 affordable_units = int(remaining_funds // (cur_ltp * (1 + buffer_pct))) if cur_ltp and cur_ltp > 0 else 0
#                 affordable_lots = affordable_units // lot_size
#                 if affordable_lots <= 0:
#                     write_log(f"Not enough remaining usable funds for any lot of {sym}. remaining_usable=₹{remaining_funds:.2f}")
#                     chunk = 0
#                     break
#                 new_chunk = affordable_lots * lot_size
#                 if new_chunk < chunk:
#                     write_log(f"Reducing chunk for {sym} from {chunk} to {new_chunk} due to remaining funds.")
#                     chunk = new_chunk
#                     continue
#                 else:
#                     chunk = 0
#                     break

#     # summary for this symbol
#     if executed_total > 0:
#         aggregated_qty = executed_total
#         aggregated_avg_price = weighted_sum / executed_total if executed_total > 0 else (ltp_cache.get(sym) or 0.0)
#         current_sl = round(aggregated_avg_price * (1 - stoploss_percent), 1)
#         try:
#             sl_entries = place_split_sl_orders(sym, aggregated_qty, current_sl, lot_size, globals().get('max_freeze_qty_per_order', DEFAULT_MAX_FREEZE_QTY))
#             write_log(f"Placed {len(sl_entries)} SL chunks for {sym}")
#         except Exception as e:
#             sl_entries = []
#             write_log(f"ERROR placing split SL for {sym}: {e}")
#             print(f"⚠️ Could not place split SL for {sym}: {e}")
#         sl_ids = [e.get("order_id") for e in sl_entries]
#         open_positions_registry[sym] = {"qty": aggregated_qty, "exchange": exchange_prefix, "sl_order_ids": sl_ids}
#         positions[sym] = {
#             "quantity": aggregated_qty,
#             "avg_buy_price": aggregated_avg_price,
#             "sl_orders": sl_entries,
#             "current_sl": current_sl,
#             "lot_size": lot_size,
#             "sub_orders": sub_order_records
#         }
#         write_log(f"AGGREGATED BUY {sym} qty={aggregated_qty} avg_price={aggregated_avg_price} SL={current_sl} SL_chunks={len(sl_entries)}")
#         print(f"Bought {sym} qty={aggregated_qty} avg_price={aggregated_avg_price}")
#     else:
#         write_log(f"No executed qty for {sym} after attempts.")

# # Post-allocation validation
# if not positions:
#     write_log("No purchases made. Exiting.")
#     raise SystemExit("⚠️ No purchases were made with provided funds / available LTPs. Exiting.")

# write_log(f"Allocation complete. Remaining usable funds ₹{remaining_funds:.2f}")
# print("\nAllocation complete. Entering monitoring phase.")
# write_log("Beginning monitoring of positions.")

# # ----------------------------
# # Monitoring loop: trailing SL, SL-miss, exit checks
# # ----------------------------
# print("\nMonitoring positions... (press Ctrl+C / STOP to trigger emergency exit)")
# log_buffer = []

# try:
#     while True:
#         all_closed = True
#         # iterate copy of keys to allow pop inside loop
#         for sym in list(positions.keys()):
#             state = positions.get(sym, {})
#             # check if kite reports open position
#             try:
#                 pos_open = False
#                 pos_data = kite.positions()
#                 for p in pos_data.get("net", []):
#                     if p.get("tradingsymbol") == sym and p.get("exchange") == exchange_prefix:
#                         if p.get("quantity", 0) > 0:
#                             pos_open = True
#                             break
#                 if not pos_open:
#                     write_log(f"Position for {sym} closed (no position in kite). Removing from tracking.")
#                     positions.pop(sym, None)
#                     open_positions_registry.pop(sym, None)
#                     continue
#             except Exception as e:
#                 write_log(f"Error reading positions for {sym}: {e}")
#                 # proceed but assume still open

#             all_closed = False

#             try:
#                 ltp = get_ltp(sym)
#             except Exception as e:
#                 write_log(f"Error fetching LTP for {sym}: {e}")
#                 time.sleep(1)
#                 continue

#             profit_pct = (ltp - state["avg_buy_price"]) / state["avg_buy_price"]
#             msg = f"{sym} LTP:{ltp:.2f} Profit%:{profit_pct*100:.2f} SL:{state['current_sl']}"
#             log_buffer.append(msg)
#             log_buffer = log_buffer[-20:]

#             # show header + logs without losing initial startup prints
#             clear_output(wait=True)
#             print("=== Trading monitor ===")
#             print(f"Remaining usable funds: ₹{remaining_funds:.2f}")
#             print("Selected symbols:", selected_sorted)
#             print("\n".join(log_buffer))

#             # SL-miss: check SL chunk statuses
#             try:
#                 sl_entries = state.get("sl_orders", [])  # list of dicts
#                 sl_ids = [e.get("order_id") for e in sl_entries if e.get("order_id")]
#                 all_statuses = []
#                 for oid in sl_ids:
#                     try:
#                         hist = kite.order_history(oid)
#                         all_statuses.extend([h.get("status") for h in hist])
#                     except Exception:
#                         all_statuses.append("OPEN")
#                 sl_filled_or_cancelled = any(s in ("COMPLETE", "CANCELLED") for s in all_statuses)
#                 if (not sl_filled_or_cancelled) and ltp <= state["current_sl"]:
#                     write_log(f"SL didn't fire for {sym} and LTP <= SL. Forcing market exit.")
#                     safe_exit(sym, state.get("quantity", 0), reason="sl_miss_force_exit")
#                     positions.pop(sym, None)
#                     continue
#             except Exception as e:
#                 write_log(f"Error checking SL status for {sym}: {e}")

#             # Trailing SL: increase SL if required
#             try:
#                 if profit_pct >= trail_start_percent:
#                     trail_step_value = state["avg_buy_price"] * trail_step_percent
#                     new_sl = round(ltp - trail_step_value, 1)
#                     if new_sl > state["current_sl"]:
#                         write_log(f"Trailing SL update for {sym}: {state['current_sl']} -> {new_sl}")
#                         sl_entries = state.get("sl_orders", [])
#                         modify_results = modify_sl_list(sl_entries, new_sl, sym)
#                         state["current_sl"] = new_sl
#             except Exception as e:
#                 write_log(f"Trailing logic failed for {sym}: {e}")

#             # Target square-off: if enabled and reached, cancel SLs and sell (we'll keep same logic but user said no button)
#             try:
#                 if globals().get("enable_square_off", True) and profit_pct >= square_off_target_percent:
#                     write_log(f"Square-off target reached for {sym} ({profit_pct*100:.2f}%). Cancelling SLs and selling all positions.")
#                     # cancel all SLs
#                     for sname, sstate in list(positions.items()):
#                         try:
#                             s_sl_entries = sstate.get("sl_orders", [])
#                             cancel_sl_list([e.get("order_id") for e in s_sl_entries if e.get("order_id")])
#                             write_log(f"Cancelled SLs for {sname} as part of square-off.")
#                         except Exception as e:
#                             write_log(f"Error cancelling SL for {sname}: {e}")
#                     time.sleep(0.5)
#                     # sell all
#                     for sname, sstate in list(positions.items()):
#                         try:
#                             qty = sstate.get("quantity", 0)
#                             if qty <= 0:
#                                 continue
#                             safe_exit(sname, qty, reason="square_off")
#                             write_log(f"Square-off sells for {sname} executed.")
#                         except Exception as e:
#                             write_log(f"Error during square-off sell for {sname}: {e}")
#                     # after square-off continue monitoring until positions close
#                     continue
#             except Exception as e:
#                 write_log(f"Error in square-off block: {e}")

#         if all_closed:
#             write_log("All positions closed. Exiting monitor.")
#             break

#         time.sleep(3)  # monitoring frequency (shorter for faster response)

# except KeyboardInterrupt:
#     write_log("KeyboardInterrupt caught - calling emergency exit.")
#     emergency_exit_handler(None, None)

# finally:
#     write_log("🔒 Strategy stopped safely.")
#     print("🔒 Strategy stopped safely.")